# Import Statements

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

## Load and Prepare Data

#### Format of the data
- last column of the data frame must contain the label and it must also be called "label"
- there should be no missing values in the data frame

In [2]:
df = pd.read_csv("heart.csv")
df = df.drop("famhist", axis=1)
df = df.rename(columns={"chd": "label"})

In [3]:
df.head()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,label
0,160,12.00,5.73,23.11,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,60,25.99,57.34,49,1


## Train-Test-Split

In [4]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [5]:
random.seed(0)
train_df, test_df = train_test_split(df, test_size=20)

## Helper Functions

In [6]:
data = train_df.values
data[:5]

array([[1.600e+02, 1.200e+01, 5.730e+00, 2.311e+01, 4.900e+01, 2.530e+01,
        9.720e+01, 5.200e+01, 1.000e+00],
       [1.440e+02, 1.000e-02, 4.410e+00, 2.861e+01, 5.500e+01, 2.887e+01,
        2.060e+00, 6.300e+01, 1.000e+00],
       [1.180e+02, 8.000e-02, 3.480e+00, 3.228e+01, 5.200e+01, 2.914e+01,
        3.810e+00, 4.600e+01, 0.000e+00],
       [1.700e+02, 7.500e+00, 6.410e+00, 3.803e+01, 5.100e+01, 3.199e+01,
        2.426e+01, 5.800e+01, 1.000e+00],
       [1.340e+02, 1.360e+01, 3.500e+00, 2.778e+01, 6.000e+01, 2.599e+01,
        5.734e+01, 4.900e+01, 1.000e+00]])

## Data pure?

In [7]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

## Classify

In [8]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

## Potential splits?

In [9]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):        # excluding the last column which is the label
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

## Split Data

In [10]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    
    return data_below, data_above

## Lowest Overall Entropy?

In [11]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [12]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [13]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

## Decision Tree Algorithm

### Representation of the Decision Tree

In [14]:
example_tree = {'age <= 30.5': [{'alcohol <= 17.305': [0.0, {'ldl <= 4.175': [0.0, 1.0]}]},
                                {'age <= 50.5': [{'typea <= 68.5': [0.0, 1.0]},
                                                 {'ldl <= 2.44': [0.0, 1.0]}]}]}

## Algorithm

In [22]:
def decision_tree_algorithm(df, counter=0, min_samples=150, max_depth=None):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # instantiate sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [23]:
tree = decision_tree_algorithm(train_df, max_depth=3)
pprint(tree)

{'age <= 30.5': [0.0,
                 {'age <= 50.5': [{'typea <= 68.5': [0.0, 1.0]},
                                  {'ldl <= 2.44': [0.0, 1.0]}]}]}


## Classification

In [24]:
example = test_df.iloc[0]
example

sbp                         136
tobacco                       0
ldl                           4
adiposity                 19.06
typea                        40
obesity                   21.94
alcohol                    2.06
age                          16
label                         0
classification                0
classification_correct     True
Name: 432, dtype: object

In [25]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [26]:
classify_example(example, tree)

0.0

## Calculate Accuracy

In [27]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [28]:
accuracy = calculate_accuracy(test_df, tree)
accuracy

0.55